# 3.9.1 同性婚に関する意見の変化再考

## 問題1
- 研究1のコントロールグループについて、第1波と第2波の感情温度の相関をみてみなはれ
    - 欠損のない観察のみを用いてやってね
- 結果について簡潔に実質的な解釈を述べなはれ

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
font = {"family":"Ricty"}
mpl.rc('font', **font)

# 研究1のコントロールグループについて、第1波と第2波の感情温度の相関をみてみなはれ
# 欠損のない観察のみを用いてやってね
import scipy.stats
gayreshaped = pd.read_csv("officialSource/qss-master/MEASUREMENT/gayreshaped.csv")
study1 = gayreshaped[gayreshaped["study"]==1]
study1_noNull = study1[(~study1[["therm1", "therm2"]].isnull().any(axis=1)) & (study1["treatment"]=="No Contact")]
# 相関係数出してみる
scipy.stats.pearsonr(study1_noNull["therm1"], study1_noNull["therm2"])
# ちなみに、NaNがある状態で相関係数だそうとしても、scipy.stats.pearsonrだとnanになる
scipy.stats.pearsonr(study1["therm1"], study1["therm2"])

相関係数＝0.9975816546030554, p値＝0.0と、相関しているというかほぼ同一なのでは？という疑惑。

In [ ]:
study2 = gayreshaped[gayreshaped["study"]==2]
study2_controll = study2[study2["treatment"]=="No Contact"]
study2_controll.corr()

問題1と同様、いずれの波でも高い相関係数となっている。

## 問題3
- 外れ値の確認、および、第1波とそれぞれの波の変化の様子がわかる散布図を描いてみなはれ

In [ ]:
# 2×2に sns.jointplotを並べようとしたけど、うまく描画できなかったので諦める
# fig = plt.figure(figsize=(16,12))
# あ、こういう書き方できるんだ！！
# [[ax1, ax2], [ax3, ax4]] = fig.subplots(nrows=2, ncols=2)

# jointplotで戻ってくるのは、JointGridオブジェクトが返ってくる。
# xxxGrid 系は、seaborn内部で複数のaxesをFigureオブジェクト内に配置してくれるやつで、
# JointGridも、X軸とY軸の上に出ている分布図も、そういう大きさのaxesを配置してくれている。
# なので、JointGrid全体のタイトルをつける場合、全体のタイトルに見えやすい位置に配置されるタイトルとして
# jointgrid.ax_marg_x のタイトルに名前を付ける
jointgrid = sns.jointplot(data=study2_controll, x="therm1", y="therm2", kind="reg")
jointgrid.ax_joint.axes.title.set_text("")
jointgrid.ax_marg_x.title.set_text("第1波と第2波の分布比較")
jointgrid.fig.show()

jointgrid2 = sns.jointplot(data=study2_controll, x="therm1", y="therm3", kind="reg")
jointgrid2.ax_marg_x.title.set_text("第1波と第3波の分布比較")
jointgrid2.fig.show()

jointgrid3 = sns.jointplot(data=study2_controll, x="therm1", y="therm4", kind="reg")
jointgrid3.ax_marg_x.title.set_text("第1波と第4波の分布比較")
jointgrid3.fig.show()

## 問題4
- CCAPのデータセットと酷似している事を見出した研究者3人の足跡を追ってみよう
    - 2012年CCAPの感情温度、研究1第1波の感情温度、研究2の第1波の感情温度のヒストグラムを見てみる
    - CCAPには多くの欠損値が含まれている。同性婚研究において、欠損データがどのように記録されたか検討しよう

In [ ]:
ccap2012 = pd.read_csv("officialSource/qss-master/MEASUREMENT/ccap2012.csv")
figure = plt.figure(figsize=(24,6))
ax1, ax2, ax3 = figure.subplots(nrows=1, ncols=3)
# 研究1第1波のヒストグラム
ax1 = sns.distplot(study1["therm1"], ax=ax1, bins=20, norm_hist=True)
ax1.set_ylim(0, 0.07)
ax2 = sns.distplot(study2["therm1"], ax=ax2, bins=20, norm_hist=True)
ax2.set_ylim(0, 0.07)
ax3 = sns.distplot(ccap2012[~ccap2012["gaytherm"].isnull()]["gaytherm"], ax=ax3, bins=20, norm_hist=True)
ax3.set_ylim(0, 0.07)
figure.show()

# CCAPのデータ分布と、今回のデータ、真ん中の分布が多いだけで、後はほとんど一緒だな・・・
# ど真ん中、つまりtherm値が50の人を、CCAPのデータから水増ししてんじゃね・・疑惑

# therm値50の人の割合を見てみると、そこだけで2割弱。飛び抜けて多いのは怪しい・・・
(study1["therm1"]==50).mean()
(study2["therm1"]==50).mean()

# CCAP の欠損データを、50で埋めたら似た分布になったりしてね
ccap2012_fillna_with50 = ccap2012.copy(deep=True)
ccap2012_fillna_with50.loc[ccap2012["gaytherm"].isnull(), "gaytherm"] = 50
ax4 = sns.distplot(ccap2012_fillna_with50["gaytherm"], bins=20, norm_hist=True)
ax4.figure.show()
# はいそっくりになったーーーー。
# なので、CCAPの欠損データをすべて50で埋めて作られていると思われる

## 問題5
- 問題4で扱ったデータを、QQプロット(的なん)で可視化してみなはれ。多分キレイに一致してるから。

In [ ]:
# 累積分布を、ｘ軸をポジション化(分位値化)してやるやーつ
import probscale
position1, values1 = probscale.plot_pos(study1["therm1"])
position2, values2 = probscale.plot_pos(study2["therm1"])
position3, values3 = probscale.plot_pos(ccap2012_fillna_with50["gaytherm"])

temp1 = pd.DataFrame({"x":position1, "y":values1, "hue":["study1" for i in range(1, len(position1) + 1)]})
temp2 = pd.DataFrame({"x":position2, "y":values2, "hue":["study2" for i in range(1, len(position2) + 1)]})
temp3 = pd.DataFrame({"x":position3, "y":values3, "hue":["ccap" for i in range(1, len(position3) + 1)]})

ax = sns.scatterplot(x="x", y="y", style="hue", hue="hue", data=temp1.append(temp3).reset_index(drop=True))
ax.figure.show()

ax = sns.scatterplot(x="x", y="y", hue="hue", data=temp2.append(temp3).reset_index(drop=True))
ax.figure.show()

# ほとんど丸かぶりで、データ生成法としての仮説は正しいだろう。

# 3.9.2 中国とメキシコにおける政治的有効性感覚

## 問題1
- 中国とメキシコの自己評価回答の分布を別々に棒グラフにせよ
    - 縦軸には回答者の割合
- それぞれお国の回答の平均を計算しまひょ
_ 上記結果から、どちらの国のほうが世辞的有効性感覚が高いといえるか？
    - メキシコは、70年以上統治してきた制度的革命党を2000年の選挙で政権から追い出し、他方中国は現在まで公正な選挙が行われたことはない。
    - この事実は、分析結果と合致するか？

In [ ]:
vignettes = pd.read_csv("officialSource/qss-master/MEASUREMENT/vignettes.csv")
myfig = plt.figure(figsize=(16,6))
ax1, ax2 = myfig.subplots(nrows=1, ncols=2)
ax1 = sns.distplot(vignettes[vignettes["china"]==0]["self"], kde=False, ax=ax1, bins=[0,1,2,3,4,5,6], norm_hist=True)
ax2 = sns.distplot(vignettes[vignettes["china"]==1]["self"], kde=False, ax=ax2, bins=[0,1,2,3,4,5,6], norm_hist=True)
ax1.set_ylim(0, 0.6)
ax2.set_ylim(0, 0.6)
ax1.title.set_text("メキシコの自己評価分布")
ax2.title.set_text("中国の自己評価分布")
myfig.show()

# それぞれの平均値
# メキシコ
vignettes[vignettes["china"]==0]["self"].mean()
# 中国
vignettes[vignettes["china"]==1]["self"].mean()


分布的にはメキシコの方が政治的有効性の自己評価は低い側に偏っており、中国は比較的平ら。
平均値も、メキシコが 1.8253、中国が 2.6219 と中国の方が高い。
政治的背景を考えると、メキシコの方が高いと予想していたが、予想とは違う結果となっている。

## 問題2
両国の政治的有効性感覚の程度の違いが、年齢分布に起因するかどうかを検討する。
- メキシコと中国の回答者の年齢ヒストグラムを作成しましょ
    - 回答者の年齢の中央値を示す縦線を描いておきましょ
- 2国間の年齢分布比較するためにQQプロット描いてみましょ
- それぞれのグラフを解釈しながら、考察してみな

In [ ]:
# 各国の年齢分布を描いてみる
figure = plt.figure(figsize=(16, 6))
ax1, ax2 = figure.subplots(nrows=1, ncols=2)
sns.distplot(vignettes[vignettes["china"]==0]["age"], 
             ax=ax1, kde=False, norm_hist=True,
             bins=[0,10,20,30,40,50,60,70,80,90,100]
             )
sns.distplot(vignettes[vignettes["china"]==1]["age"],
             ax=ax2, kde=False, norm_hist=True,
             bins=[0,10,20,30,40,50,60,70,80,90,100]
             )
ax1.title.set_text("メキシコの年齢分布")
ax2.title.set_text("中国の年齢分布")
ax1.set_ylim(0,0.05)
ax2.set_ylim(0,0.05)
# 中央値の垂直線を追加
ax1.axvline(x=vignettes[vignettes["china"]==0]["age"].median())
ax1.text(x=vignettes[vignettes["china"]==0]["age"].median(), y=0.03, s="中央値", ha="left", va="center")
ax2.axvline(x=vignettes[vignettes["china"]==1]["age"].median())
ax2.text(x=vignettes[vignettes["china"]==1]["age"].median(), y=0.03, s="中央値", ha="left", va="center")
figure.show()
# ちなみに中央値は・・メキシコが35歳、中国が45歳
vignettes[vignettes["china"]==0]["age"].median()
vignettes[vignettes["china"]==1]["age"].median()

# QQプロット的なものを描く
import probscale
mexico_pos, mexico_val = probscale.plot_pos(vignettes[vignettes["china"]==0]["age"])
china_pos, china_val = probscale.plot_pos(vignettes[vignettes["china"]==1]["age"])

temp1 = pd.DataFrame(data={"x":mexico_pos, "y":mexico_val, "country":["mexico" for i in range(1, len(mexico_pos)+1 )]})
temp2 = pd.DataFrame(data={"x":china_pos, "y":china_val, "country":["china" for i in range(1, len(china_val)+1 )]})
temp = temp1.append(temp2).reset_index(drop=True)

fig = plt.figure()
ax = fig.subplots()
sns.scatterplot(ax=ax, x="x", y="y", hue="country", data=temp, alpha=0.3)
ax.figure.show()

ヒストグラムで見ると、メキシコの年齢層は20代がもっとも多く、中国は30代、40代が中心。(中央値はそれぞれ35歳、45歳)
QQプロット的なものではなんとも言えないが、ヒストグラムと中央値からは中国側がやや年齢層が高い事がわかり、この事が前問の予想とのギャップを生んでいる可能性がある。

## 問題3
回答者によって、質問の捉え方が違う可能性がある。例えば、自己評価を「かなりある」と答えた2人でも、その2人が極めて異なる政治状況に直面していた場合、「かなりある」が表す程度が違うのでは？という可能性がある。(現実問題として、政治に対する介入力をもっていんたい中国国民がいう「かなりある」は、政治に対する介入力があるメキシコ国民の「かなりある」よりも表す程度が低い可能性がある)

- その影響を除外するために、ヴィニエット形式問題の最後の設問「モーゼズ」の回答よりも、自己評価を低く答えた回答者の割合を、国ごとに求めよ。
- 前問と比較して、今回の結果の解釈を簡潔に述べよ


In [ ]:
# モーゼス回答よりも低い自己評価を答えた人の割合を、国別に出す
vignettes["SelfIsLowerThanMoses"] = vignettes["self"] < vignettes["moses"]
SelfIsLowerThanMoses = vignettes.groupby("china").sum()[["SelfIsLowerThanMoses"]]
SelfIsLowerThanMoses["count"] = vignettes.groupby("china").count()["SelfIsLowerThanMoses"]
SelfIsLowerThanMoses["ratio"] = SelfIsLowerThanMoses['SelfIsLowerThanMoses'] / SelfIsLowerThanMoses['count']
SelfIsLowerThanMoses["ratio"]

問題1では、メキシコの自己評価が中国より低い方に偏っている結果だったが、モーゼスの評価よりも自己評価が低い割合は中国が0.561837, メキシコが0.248996と倍以上の差が開いた。
この事から、モーゼスのような「社会に対してなんの力もない」という状況を基準にした相対的な自己評価の高低では、メキシコでは自己評価が高い人の割合が8割弱、中国は4割強となり、中国のほうが自己評価が低いと考えられる。

模範解答では・・・

中国は、主観的には「政治に対して介入力がある」と感じているが、モーゼスの状況よりも自己評価が低いと感じている。

In [ ]:
temp = vignettes[(vignettes["alison"] >= vignettes["jane"]) &(vignettes["jane"] >= vignettes["moses"])].copy()
# 境界値が等しい場合の処理については、模範解答が高い方に寄せてあったのでそれに倣う
temp.loc[temp['alison'] <= temp['self'], "selfPosition"] = 4
temp.loc[(temp['jane'] <= temp['self']) & (temp['self'] < temp['alison']), "selfPosition"] = 3 
temp.loc[(temp['moses'] <= temp['self']) & (temp['self'] < temp['jane']), "selfPosition"] = 2 
temp.loc[temp['self'] < temp['moses'], "selfPosition"] = 1

myFig = plt.figure(figsize=(16,6))
ax1, ax2 = myFig.subplots(ncols=2,nrows=1)
ax1 = sns.distplot(temp.loc[temp['china']==0, 'selfPosition'], bins=[1,2,3,4,5], norm_hist=True, kde=False, ax=ax1)
ax1.title.set_text('メキシコの分布')
ax2 = sns.distplot(temp.loc[temp['china']==1, 'selfPosition'], bins=[1,2,3,4,5], norm_hist=True, kde=False, ax=ax2)
ax2.title.set_text('中国の分布')
ax1.figure.show()

# 平均値を取る
temp.loc[temp['china']==0, 'selfPosition'].mean()
temp.loc[temp['china']==1, 'selfPosition'].mean()
# グラフに描いちゃう
ax1.axvline(temp.loc[temp['china']==0, 'selfPosition'].mean())
ax2.axvline(temp.loc[temp['china']==1, 'selfPosition'].mean())
ax1.text(x=temp.loc[temp['china']==0, 'selfPosition'].mean(), y=0.35, s="平均値:2.93", ha="right", va="center")
ax2.text(x=temp.loc[temp['china']==1, 'selfPosition'].mean(), y=0.4, s="平均値:2.15", ha="left", va="center")
ax1.set_ylim(0,0.5)
ax2.set_ylim(0,0.5)
myFig.show()

問題1では回答の数値をそのまま集計しており、定性的な概念を数値化する際の個人感覚のブレが影響し、想定外にメキシコの自己評価が低かった。
しかし、その「ブレ」をヴィニエットで固定化して基準尺度として捉え、その尺度上で自己評価がどの位置にあるかを改めて集計すると、メキシコのほうが自己評価は高かった。これは政治的背景とも一致している。

## 問題5
問題4の状況は若者と中高年ではどちらが深刻か？
- 40歳以上と40歳未満、別々に問題4と同じ事を答えよ
- 年齢グループを2つに分ける事で、問題4での回答が変わるだろうか？
	- 設問2でわかったことと関連づけて論じなされ

In [ ]:
over40 = temp[temp['age'] >= 40]
under40 = temp[temp['age'] < 40]


myfig = plt.figure(figsize=(16,12))
[[ax1,ax2],[ax3,ax4]] = myfig.subplots(nrows=2, ncols=2)
# 上段はメキシコ
ax1 = sns.distplot(under40.loc[under40['china']==0, 'selfPosition'], ax=ax1, bins=[1,2,3,4,5], norm_hist=True, kde=False)
ax2 = sns.distplot(over40.loc[over40['china']==0, 'selfPosition'], ax=ax2, bins=[1,2,3,4,5], norm_hist=True, kde=False)
ax3 = sns.distplot(under40.loc[under40['china']==1, 'selfPosition'], ax=ax3, bins=[1,2,3,4,5], norm_hist=True, kde=False)
ax4 = sns.distplot(over40.loc[over40['china']==1, 'selfPosition'], ax=ax4, bins=[1,2,3,4,5], norm_hist=True, kde=False)
# 上段はメキシコ、左がUNDER40
ax1.title.set_text('メキシコ 40代以下')
ax2.title.set_text('メキシコ 40代以上')
ax3.title.set_text('中国 40代以下')
ax4.title.set_text('中国 40代以上')
ax1.set_ylim(0,0.6)
ax2.set_ylim(0,0.6)
ax3.set_ylim(0,0.6)
ax4.set_ylim(0,0.6)
myfig.show()

問題2では、中国とメキシコの標本の年齢分布が、中国がやや40代に偏っているので、問題4の違い(国別での自己評価に差がある)が、年齢に起因する可能性が捨てきれない。
それを確認するために、年齢別での集計(特にキーとなる40代を境界線として)して見たが、各年齢とも、問題4と同じく国別で傾向が違い、その違いは問題4と同じものだった。
なので、問題4で観察できた差は、年齢によるものではないと思われる。

# 3.9.3 国連総会における投票

## 問題1
共産主義の終焉以降、各国の理想点がどのように変化をしたのかを検討するため、以下のような手順を考えた。
- 1980年(ベルリンの壁崩壊10年前)と2000年(ベルリンの壁崩壊10年後)、それぞれの「理想点の分布」をグラフにする
- それぞれのグラフに中央値を縦線で描く
- 2つの分布はどのように異なっているだろうか？分極化の程度に注意を払い、簡潔に結果の実質的な解釈を述べよう。

In [ ]:
# データ読み込み
unvoting = pd.read_csv("officialSource/qss-master/MEASUREMENT/unvoting.csv")
myfig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
# bins作成
myBins = [i/2 for i in range(-8, 9)]
ax1 = sns.distplot(unvoting[unvoting["Year"]==1980]["idealpoint"], ax=ax1, bins=myBins, norm_hist=True)
ax1.title.set_text("1980年の理想点分布")
ax2.title.set_text("2000年の理想点分布")
ax2 = sns.distplot(unvoting[unvoting['Year']==2000]['idealpoint'], ax=ax2, bins=myBins, norm_hist=True)
ax1.set_xlim(-4,4)
ax2.set_xlim(-4,4)
ax1.set_ylim(0,1)
ax2.set_ylim(0,1)

median1980 = unvoting[unvoting["Year"] == 1980]["idealpoint"].median()
ax1.axvline(median1980)
ax1.text(x=median1980, y=0.8, s='中央値：'+ str(median1980), ha='left', va='center')
median2000 = unvoting[unvoting["Year"] == 2000]["idealpoint"].median()
ax2.text(x=median2000, y=0.7, s='中央値：'+ str(median2000), ha='left', va='center')
ax2.axvline(median2000)
myfig.show()


unvoting[unvoting["Year"] == 1980]["idealpoint"].describe()
unvoting[unvoting["Year"] == 2000]["idealpoint"].describe()

# ボックスプロットする
bpfigure, ax1 = plt.subplots()
boxplot_df = unvoting.loc[(unvoting["Year"] == 1980) | (unvoting["Year"] == 2000), ['Year', "idealpoint"]]
ax1 = sns.boxplot(x='Year', y='idealpoint', data=boxplot_df, ax=ax1)
bpfigure.show()

ベルリンの壁崩壊前後の10年、崩壊後には確かに分布がリベラル側に広がっているが、中央値値じたいは共産主義側に動いた。
四分位で見ても、75％値はリベラル側に広がっている。さらに、分布図では山が2つあるような状態で、集団としては2極化が進んでいる可能性がある。

## 問題2
アメリカ支持の国々がどのように変化(推移)したかを検討するため、以下の手順で調査を行う
- 全加盟国におけるアメリカとの一致割合の平均値が時間とともにどのように変化したかをグラフ化
- 比較用に、ロシアとの一致割合の平均を表す線も描く
	- アメリカの孤立は、ロシアと比べて次第に強まっているだろうか？弱まっているだろうか？
- 一貫してアメリカ支持の国を特定しよう
- 最もロシア支持の国はどこかも特定しよう
- 結果について簡潔に実質的な解釈を述べよう

In [ ]:
# データ準備
# アメリカとロシアを除外するために国名を確認
print(unvoting.groupby(['CountryName', 'CountryAbb']).count())
# 一旦アメリカとロシアを除外
ts_df = unvoting[(unvoting['CountryAbb']!='USA') & (unvoting['CountryAbb']!='RUS')].copy()
# 年ごとに、PctAgreeXXの平均を算出
ts_df_graph = ts_df.groupby(by='Year').mean()[['PctAgreeUS', 'PctAgreeRUSSIA']]
ts_df_graph= ts_df_graph.stack().reset_index().rename(columns={'level_1': 'hue', 0: 'mean'})
# グラフ化
ts_fig, ax1 = plt.subplots(ncols=1, nrows=1, figsize=(8, 6))
ax1 = sns.lineplot(x='Year', y='mean', hue='hue', style='hue', data=ts_df_graph, dashes=False, markers=['o','o'], ax=ax1)
ts_fig.show()

In [ ]:
# 一貫してアメリカ支持の国、ロシア支持の国をリストアップ
ts_df.loc[unvoting['PctAgreeUS'] > unvoting['PctAgreeRUSSIA'], 'USAAgreeCountry'] = 1
ts_df.loc[unvoting['PctAgreeUS'] == unvoting['PctAgreeRUSSIA'], 'USAAgreeCountry'] = np.nan
ts_df.loc[unvoting['PctAgreeUS'] < unvoting['PctAgreeRUSSIA'], 'USAAgreeCountry'] = 0
ts_df_USAagree = ts_df.groupby(by=['CountryName','USAorRUS'], as_index=False).count()['CountryName', ]